1. Data Type and Transformation Challenges
Challenge: SalePrice was initially in an object (string) format, causing issues in model compatibility and scaling.
Solution: We converted SalePrice to a numeric type and applied a log transformation (np.log1p) to stabilize its variance. This transformation compresses the wide range of house prices, making it easier for the model to train.
2. Feature Selection
Challenge: Not all features in the dataset were equally useful. Some features (like YearBuilt, LandUse) had low correlation with SalePrice, which could introduce noise.
Solution: We identified and retained only the high-correlation features (TotalValue, LandValue, BuildingValue, FullBath, Bedrooms, Acreage) to improve the model's focus on relevant information.
3. Categorical Encoding
Challenge: Categorical features (LandUse, SoldAsVacant) needed encoding for the neural network, as it requires numerical input.
Solution: We applied One-Hot Encoding for categorical columns, which allowed us to treat these features without implying any ordinal relationship.
4. Scaling
Challenge: Neural networks are sensitive to feature scales, so unscaled data could lead to poor convergence.
Solution: We used StandardScaler to normalize the numerical features in X, which helps the model learn more effectively.
5. Model Convergence and High MAE
Challenge: Initial model training showed very high Mean Absolute Error (MAE) and unstable learning, possibly due to the learning rate being too high.
Solution: We progressively reduced the learning rate, finally setting it to 0.00001 for more stable convergence. This adjustment improved the training stability and reduced MAE significantly over successive epochs.
6. Interpreting Predictions on Original Scale
Challenge: Since we log-transformed SalePrice, we had to carefully interpret predictions on the original scale.
Solution: After model predictions, we applied the inverse transformation (np.expm1) to convert predictions back to their original scale, allowing us to compare directly with actual prices.
7. High Discrepancies in Predictions (After Inverse Transformation)
Challenge: Despite a low MAE in the transformed space, predictions on the original scale showed significant discrepancies for high-value properties.
Solution: We suggested further adjustments like trying alternative transformations (e.g., quantile transformation), experimenting with different loss functions (e.g., MAPE), and considering ensemble models like XGBoost for potentially better handling of the price range.
Summary of Key Changes
Converted SalePrice to numeric and applied log transformation.
Selected high-correlation features and encoded categorical data.
Scaled features with StandardScaler.
Tuned the model’s learning rate to improve convergence.
Interpreted predictions by applying the inverse of the log transformation.
These steps collectively improved the model's performance and addressed the initial challenges effectively. Let me know if you'd like more details on any specific step!

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Load and preprocess data
nashville_data = pd.read_csv('Nashville_Housing_ML.csv')

# Apply log transformation to SalePrice for stability
nashville_data['SalePrice'] = np.log1p(nashville_data['SalePrice'])

# Drop irrelevant columns
nashville_data = nashville_data.drop(['SaleDate', 'OwnerName'], axis=1)

# One-hot encode categorical columns
nashville_data = pd.get_dummies(nashville_data, columns=['LandUse', 'SoldAsVacant'], drop_first=True)

# Select high-correlation features
selected_features = ['TotalValue', 'LandValue', 'BuildingValue', 'FullBath', 'Bedrooms', 'Acreage']
X = nashville_data[selected_features].values
y = nashville_data['SalePrice'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the ANN model with a smaller learning rate for stability
ann = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear')
])

ann.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='mean_squared_error', metrics=['mae'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Train the model
ann.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping])

# Predict and apply inverse transformation if log-transformed
raw_predictions = ann.predict(X_test)
predictions = np.expm1(raw_predictions)  # Use np.expm1 if SalePrice was log-transformed


Epoch 1/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 144.4910 - mae: 11.9862
Epoch 2/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 137.4883 - mae: 11.6913
Epoch 3/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 128.0585 - mae: 11.2747
Epoch 4/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 115.9877 - mae: 10.7172
Epoch 5/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 101.1749 - mae: 9.9842
Epoch 6/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 86.2364 - mae: 9.1640
Epoch 7/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 72.9616 - mae: 8.3346
Epoch 8/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59.5533 - mae: 7.4372
Epoch 9/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49.3616 - mae: 6.5431
Epoch 10/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 39.4645 - mae: 5.6423
Epoch 11/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 32.0415 - mae: 4.8929
Epoch 12/50
632/632 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 26.3366 - mae: 4.3242
Epoc

In [2]:
# Evaluate on test set
test_loss, test_mae = ann.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

# Revert log transformation on predictions to get actual prices
predictions = np.expm1(ann.predict(X_test))

# Display a few predictions alongside actual values
for i in range(5):
    print(f"Predicted: {predictions[i][0]:.2f}, Actual: {np.expm1(y_test[i]):.2f}")


158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.1538 - mae: 0.7685
Test MAE: 0.7575606107711792
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Predicted: 214281.83, Actual: 339000.00
Predicted: 92094.12, Actual: 139900.00
Predicted: 74330.88, Actual: 135000.00
Predicted: 118240.88, Actual: 882700.00
Predicted: 91280.58, Actual: 200000.00
